In [ ]:
import logging
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, ContextTypes
import schedule
import time
import asyncio
import threading

logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

users = set()

async def send_daily_message(context: ContextTypes.DEFAULT_TYPE):
    for user_id in users:
        await context.bot.send_message(chat_id=user_id, text="Привет! Не забудь заполнить опросник по самочувствию: https://docs.google.com/forms/d/e/1FAIpQLSftzDg_FIBpE-A6yP9grZzGRTWpjk9tVALxUWteEz2LGc-CLA/viewform?vc=0&c=0&w=1&flr=0")

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    user_id = update.message.chat_id
    users.add(user_id)
    await update.message.reply_text("Привет! Это бот, который напоминает о том, что нужно заполнять опросник по самочувствию. Теперь каждое утро в 11 часов ты будешь получать сообщение со ссылкой")

def run_scheduler(application: ApplicationBuilder):
    while True:
        schedule.run_pending()
        time.sleep(1)

def schedule_daily_message(application: ApplicationBuilder):
    schedule.every().day.at("11:00").do(lambda: asyncio.run(send_daily_message(application)))
    
def main():
    application = ApplicationBuilder().token("TOKEN").build()
    application.add_handler(CommandHandler("start", start))
    threading.Thread(target=run_scheduler, args=(application,), daemon=True).start()
    schedule_daily_message(application)
    application.run_polling()

if __name__ == '__main__':
    main()